<a href="https://colab.research.google.com/github/ThinkerPcw/Colab/blob/master/reid_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 创建删除文件夹

In [2]:
!mkdir storage
!rm -r sample_data

rm: cannot remove 'sample_data': No such file or directory


# 挂载google drive,下载数据集,与自己的包

In [0]:
!rm -rf sample_data
from google.colab import drive
import os
import zipfile
drive.mount('/pcw')
market_path="/pcw/My Drive/Colab/datasets/Market-1501-v15.09.15.zip"
file_zip = zipfile.ZipFile(market_path, 'r')
for file in file_zip.namelist():
      print(file)
      file_zip.extract(file, r'.')

reid_path="/pcw/My Drive/Colab/datasets/reid.zip"
file_zip = zipfile.ZipFile(reid_path, 'r')
for file in file_zip.namelist():
      print(file)
      file_zip.extract(file, r'.')

# 从自己的库导入需要的包

In [0]:
from reid.data import datasets
from reid.data import transforms
from reid.model.spherenet import *
from reid.utils import feature_operate as FO
from reid.utils.model_save_restore import *
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from reid.evaluation import market_evaluate
from reid.data.samplers import BalancedSampler
import torch
from torch.nn import CrossEntropyLoss

# 定义数据加载

In [0]:
EPOCH = 140
LR = 0.02

train_image_path = "./Market-1501-v15.09.15/bounding_box_train"
query_path = "./Market-1501-v15.09.15/query"
gallery_path = "./Market-1501-v15.09.15/bounding_box_test"

save_path = './storage'

train_transform = transforms.Compose([
    transforms.RandomSizedRectCrop(256, 128),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_dataset = datasets.MarketDataset(image_path=train_image_path, transform=train_transform, use_onehot=False)
sampler = BalancedSampler(train_dataset, 20, 6)
train_loader = DataLoader(dataset=train_dataset, batch_sampler=sampler, num_workers=4)

test_transform = transforms.Compose([
    transforms.Resize((256, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
query_dataset = datasets.MarketDataset(image_path=query_path, transform=test_transform, use_onehot=False)
gallery_dataset = datasets.MarketDataset(image_path=gallery_path, transform=test_transform, use_onehot=False)
query_loader = DataLoader(query_dataset, batch_size=16, shuffle=False, num_workers=4)
gallery_loader = DataLoader(gallery_dataset, batch_size=16, shuffle=False, num_workers=4)

# 定义模型，并将训练好的模型保存到google drive

In [0]:
net = nn.DataParallel(spherenet50()).cuda()
print(net)


loss = CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=5e-4)


def lr_scheduler(epoch, optimizer):
    warmup_epoch = 20
    warmup_lr = 5e-5
    start_lr = 1e-3
    lr_steps = [80, 100]
    lr_factor = 0.1
    if epoch < warmup_epoch:  # warmup
        warmup_scale = (start_lr / warmup_lr) ** (1.0 / warmup_epoch)
        lr = warmup_lr * (warmup_scale ** epoch)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        optimizer.defaults['lr'] = lr
    else:
        for i, el in enumerate(lr_steps):
            if epoch == el:
                lr = start_lr * (lr_factor ** (i + 1))
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr
                optimizer.defaults['lr'] = lr
    return optimizer


step = 0
best_map = -1
best_map_epoch = 0
best_rank1 = -1
best_rank1_epoch = 0
print("开始训练>>>")
for epoch in range(EPOCH):
    optimizer = lr_scheduler(epoch, optimizer)
    for images, ids, cams in train_loader:
        predict = net(images.cuda())
        loss_value = loss(predict, ids.cuda())
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
        if step % 10 == 0:
            print(step, loss_value.item())
        step += 1
    if epoch + 1 >= 10 and (epoch + 1) % 10 == 0:
        print("第{}轮效果评估开始>>>".format(epoch + 1))
        query_feature = FO.extract_cnn_feature(net, loader=query_loader, vis=False)
        gallery_feature = FO.extract_cnn_feature(net, loader=gallery_loader, vis=False)
        query_id, query_camera = query_loader.dataset.original_id, query_loader.dataset.cameras
        gallery_id, gallery_camera = gallery_loader.dataset.original_id, gallery_loader.dataset.cameras
        map, cmc = market_evaluate.evaluate(query_feature, np.array(query_id), np.array(query_camera), gallery_feature,
                                            np.array(gallery_id), np.array(gallery_camera), vis=False)
        print("第{}轮训练结果: map:{},rank-1:{}".format(epoch + 1, map, cmc[0]))
        if map > best_map or cmc[0] > best_rank1:
            save_network(save_path, net, epoch)
            with open('./storage/net_{}.pth'.format(epoch),'r') as f,open('../pcw/My Drive/Colab/experiment/storage/net_{}.pth'.format(epoch),"w") as fw:
                 fw.write(f.read())
        if map > best_map:
            best_map = map
            best_map_epoch = epoch
        if cmc[0] > best_rank1:
            best_rank1 = cmc[0]
            best_cmc_epoch = epoch
    print("已经训练了{}个epoch".format(epoch + 1))
print("最佳map:{},最佳rank-1{},最佳map训练轮数:{},最佳cmc训练轮数:{}".format(best_map, best_rank1, best_map_epoch, best_rank1_epoch))


DataParallel(
  (module): SphereNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0)

In [0]:
10:31

In [0]:
# from google.colab import files
# files.download('./storage/net_129.pth')

In [0]:
# with open('./sample_data/README.md','r') as f,open("../pcw/My Drive/Colab/experiment/storage/hah.md","w") as fw:
#    fw.write(f.read())